In [1]:
import numpy as np
import pyci
import psi4

In [2]:
# Lets intiate AOint() a subclass of psi4utils() 
aoint = pyci.AOint('d-aug-cc-pv5z', 'He.xyz')
# saves all the necessary integrals as .npz files
aoint.save_all_aoints()
# run scf and get info for canonical HF orbitals
aoint.save_mo_info()
# lets load  ao_erints, mo_info and try to get the CIS matrix
ao_erints = np.load('ao_erints.npz')['electron_repulsion_aoints'] 
eps_a = np.load('mo_scf_info.npz')['eps_a']
Ca = np.load('mo_scf_info.npz')['Ca'] 
# lets convert our erints from AO basis to MO basis
mo_erints = aoint.eri_ao2mo(Ca, ao_erints)
del ao_erints

Size of the ERI tensor will be 0.97 Gb

Ground state SCF Energy : -2.86162717 

Nuclear repulsion energy : 0.00000000 

Total electronic energy : -2.86162717 



In [3]:
nbf, nmo, nso, na, nb, nocc, nvirt = aoint.get_orb_info(aoint.wfn)

 ORBITAL INFORMATION  
-----------------------
Basis functions   : 105  
Molecular orbitals: 105  
Spin Orbitals     : 210  
Alpha Orbitals    : 1  
Beta Orbitals     : 1  
Occupied Orbitals : 1  
Virtual Orbitals  : 104  



In [4]:
scf_energy = aoint.scf_energy
psi4_cisd = psi4.energy('CISD')

In [5]:
orbinfo = (nocc, nmo)
active_space = (nocc,nvirt)
singles = True
full_cis = True
doubles = True
doubles_options = [True, True, True, True, True]
options = singles, full_cis, doubles, doubles_options
csfs, num_csfs = pyci.generate_csfs(orbinfo, active_space, options)
num_csfs, sum(num_csfs)

([1, 104, 104, 5356, 0, 0, 0], 5565)

In [6]:
HCISD = pyci.comp_hcisd(eps_a, mo_erints, scf_energy, orbinfo, active_space, options, ncore=22)

In [8]:
import os 
os.environ['OMP_NUM_THREADS']='22'


In [9]:
vals, vecs = np.linalg.eigh(HCISD)

In [10]:
# Comparing pyci with psi4
print("pyci CISD E0: {e:16.16f}".format(e=vals[0]))
print("psi4 CISD E0: {e:16.16f}".format(e=psi4_cisd))
print("dE : {dE:1.2E}".format(dE=abs(psi4_cisd-vals[0])))


pyci CISD E0: -2.9032019441778978
psi4 CISD E0: -2.9032019441778933
dE : 4.44E-15


In [ ]:
HCISD.shape